In [1]:
import pyspark
sc = pyspark.SparkContext()

In [11]:
textfile = sc.textFile("data1.csv")

In [24]:
ventas = textfile.map(lambda x:tuple(x.split(','))).map(lambda x:(int(x[0]),int(x[1]),int(x[2])))

In [25]:
ventas.take(2)

[(2019, 158, 1651), (2018, 175, 1958)]

In [26]:
textfile2 = sc.textFile("data2.csv")

In [27]:
productos = textfile2.map(lambda x:tuple(x.split(','))).map(lambda x:(int(x[0]),x[1],x[2]))

In [28]:
productos.take(2)

[(1, 'Descripcion Random', 'Comida'), (2, 'Descripcion Random', 'Muebles')]

## Inciso A

In [29]:
cantidad_ventas = ventas.map(lambda x: (x[1],1))

In [35]:
cantidad_ventas.reduceByKey(lambda x,y:x+y).reduce(lambda x,y: x if x[1]>y[1] else y)[0]

80

## Inciso B

In [36]:
cantidad_ventas.take(2)

[(158, 1), (175, 1)]

In [38]:
categorias_por_id = productos.map(lambda x: (x[0],x[2]))
categorias_por_id.take(2)

[(1, 'Comida'), (2, 'Muebles')]

In [47]:
cantidad_ventas.join(categorias_por_id).map(lambda x:(x[1][1],x[1][0])).reduceByKey(lambda x,y:x+y).reduce(lambda x,y: x if x[1]>y[1] else y)[0]

'Tocador'

## Inciso C

In [64]:
top_vendidos = sc.parallelize(cantidad_ventas.reduceByKey(lambda x,y:x+y).takeOrdered(5,lambda x: -x[1]))

In [65]:
descripcion_por_id = productos.map(lambda x: (x[0],x[1]))

In [69]:
top_vendidos.join(descripcion_por_id).map(lambda x:(x[0],x[1][1],x[1][0])).collect()

[(80, 'Descripcion Random', 7),
 (170, 'Descripcion Random', 6),
 (104, 'Descripcion Random', 5),
 (51, 'Descripcion Random', 6),
 (141, 'Descripcion Random', 6)]